# Enrichment analysis of GPs

In [ ]:
import os
import csv
import numpy as np
import pandas as pd
import gseapy as gp
import matplotlib.pyplot as plt
from tabulate import tabulate
import re
import seaborn as sns
from matplotlib.colors import LogNorm

# Set parameter

In [ ]:
Top_n_gps = 10
gp_compare_type = '1to1' ##'1rest' '1to1'
gp_bayes_direction = 'up' #'down' , 'up' or 'abs'
Top_n_ptw = 10

output_path = '/Users/yelin.zhao/Library/CloudStorage/OneDrive-Personal/Projects/Spatial/Spatial_CCC/artifacts/sample_integration/Joakim_prostate_H21_H14_H15_H25_epochs400_svg2000_m6/GP_path_enrichment'
os.makedirs(output_path, exist_ok=True)

# Get gene list

In [ ]:
if gp_compare_type == '1rest':
    GP_per_niche = pd.read_csv('/Users/yelin.zhao/Library/CloudStorage/OneDrive-Personal/Projects/Spatial/Spatial_CCC/artifacts/sample_integration/Joakim_prostate_H21_H14_H15_H25_epochs400_svg2000_m6/figures/log_bayes_factor_2.3_GP_per_niche_results.csv', index_col = 0)
elif gp_compare_type == '1to1':
    GP_per_niche = pd.read_csv('/Users/yelin.zhao/Library/CloudStorage/OneDrive-Personal/Projects/Spatial/Spatial_CCC/artifacts/sample_integration/Joakim_prostate_H21_H14_H15_H25_epochs400_svg2000_m6/figures/log_bayes_factor_2.3_GP_per_comp_results.csv', index_col = 0)

GP_per_niche


In [ ]:
Active_GPs_summary = pd.read_csv('/Users/yelin.zhao/Library/CloudStorage/OneDrive-Personal/Projects/Spatial/Spatial_CCC/artifacts/sample_integration/Joakim_prostate_H2_1_H1_4_H1_5_H_2_5_epochs400_svg5000_m3/figures/Active_GPs_summary.csv', index_col = 0)
Active_GPs_summary

In [ ]:
# Extract top N GPs from each comparison
GP_per_niche = GP_per_niche[~GP_per_niche['gene_program'].str.startswith('Add-on_')] #uncomment this line if want to exclude 'Add-on_GP'
GP_per_niche['abs_log_bayes_factor'] = GP_per_niche['log_bayes_factor'].abs()
if gp_bayes_direction == 'up':
    GP_per_niche = GP_per_niche[GP_per_niche['log_bayes_factor'] > 0]
    GP_per_niche_sorted = GP_per_niche.sort_values(by='abs_log_bayes_factor', ascending=False)
elif gp_bayes_direction == 'down':
    GP_per_niche = GP_per_niche[GP_per_niche['log_bayes_factor'] < 0]
    GP_per_niche_sorted = GP_per_niche.sort_values(by='abs_log_bayes_factor', ascending=False)
elif gp_bayes_direction == 'abs':
    GP_per_niche_sorted = GP_per_niche.sort_values(by='abs_log_bayes_factor', ascending=False)

# Group by comparison_pair and select the top 10 rows within each group
top_df = GP_per_niche_sorted.groupby('comparison_pair').head(Top_n_gps)
top_df = top_df.sort_values(by=['comparison_pair', 'log_bayes_factor'], ascending=[True, False])
top_df.reset_index(drop=True, inplace=True)

top_df.to_csv(f"{output_path}/{gp_compare_type}_TopGPs_{gp_bayes_direction}.csv", index = False)

In [ ]:
top_df

In [ ]:
Top_GPs_summary = pd.merge(Active_GPs_summary, top_df, left_on='gp_name', right_on='gene_program', how='inner')
Top_GPs_summary

In [ ]:
Top_GPs_summary = pd.merge(Active_GPs_summary, top_df, left_on='gp_name', right_on='gene_program', how='inner')
Top_GPs_summary = Top_GPs_summary.sort_values(by=['comparison_pair', 'abs_log_bayes_factor'], ascending=[True, False])
Top_GPs_summary.reset_index(drop=True, inplace=True)
Top_GPs_summary

# Pathway Enrichment
## Enrichment of functional pathways

In [ ]:
# default: Human
names = gp.get_library_name()
names 
#'GWAS_Catalog_2023','UK_Biobank_GWAS_v1','GO_Biological_Process_2023', 'MSigDB_Hallmark_2020','Reactome_2022', KEGG_2021_Human IDG_Drug_Targets_2022'  'MSigDB_Oncogenic_Signatures',  'HMDB_Metabolites','Metabolomics_Workbench_Metabolites_2022',

In [ ]:
from IPython.display import display

GO_enrichemnt = list()
for comparison_pair in Top_GPs_summary['comparison_pair'].unique():
    print(comparison_pair)
    
    original_string = ''  
    df_subset = Top_GPs_summary[Top_GPs_summary['comparison_pair'] == comparison_pair]
    df_subset.reset_index(drop=True, inplace=True)
    for index, row in df_subset.iterrows():
        original_string = original_string + df_subset['gp_source_genes'][0]
        original_string = original_string + df_subset['gp_target_genes'][0]
    #string_list = re.findall(r"'(.*?)'", original_string)
    string_list = list(set(re.findall(r"'(.*?)'", original_string)))  # Convert to set to remove duplicates
    print(string_list)
    
    enr_KEEG = gp.enrichr(gene_list=string_list,
                          gene_sets=['MSigDB_Hallmark_2020','KEGG_2021_Human','GO_Biological_Process_2023','Reactome_2022'],#'GO_Biological_Process_2023','MSigDB_Hallmark_2020','KEGG_2021_Human'
                          organism='Human').results
    #'Reactome_2022', KEGG_2021_Human IDG_Drug_Targets_2022' 'MSigDB_Hallmark_2020','MSigDB_Oncogenic_Signatures', 'Metabolomics_Workbench_Metabolites_2022',

    summary = enr_KEEG[enr_KEEG['Adjusted P-value'] < 0.05][['Term', 'Adjusted P-value','Gene_set','Overlap','Genes']]
    summary['comparison_pair'] = comparison_pair
    
    GO_enrichemnt.append(summary)


In [ ]:
GO_enrichemnt = pd.concat(GO_enrichemnt)

# remove duplicate terms
GO_enrichemnt= GO_enrichemnt[GO_enrichemnt['Adjusted P-value'] < 0.05]
GO_enrichemnt = GO_enrichemnt.sort_values(by=['comparison_pair', 'Adjusted P-value'], ascending=[True, True])
GO_enrichemnt = GO_enrichemnt.drop_duplicates(subset='Term', keep='first')  # Keep the first occurrence of each unique value in 'term'
GO_enrichemnt.reset_index(drop=True, inplace=True)

GO_enrichemnt_top = GO_enrichemnt.groupby('comparison_pair').head(Top_n_ptw)
GO_enrichemnt_top.reset_index(drop=True, inplace=True)

GO_enrichemnt.to_csv(f"{output_path}/Pathway_enrichment_{gp_compare_type}_{gp_bayes_direction}.csv", index = False)
GO_enrichemnt_top.to_csv(f"{output_path}/Pathway_enrichment_{gp_compare_type}_top{Top_n_ptw}pathway_{gp_bayes_direction}.csv", index = False)


In [ ]:
GO_enrichemnt_top

In [ ]:
GO_enrichemnt_top_wide = GO_enrichemnt_top.pivot(index='Term', columns='comparison_pair', values='Adjusted P-value')
if gp_compare_type == '1to1':
    GO_enrichemnt_top_wide = GO_enrichemnt_top_wide.reindex(columns=['1 vs 5','1 vs 2','1 vs 0','1 vs 4','1 vs 3'])#['2 vs 4','2 vs 1','2 vs 3','2 vs 0','2 vs 5']

GO_enrichemnt_top_wide = GO_enrichemnt_top_wide.replace(np.nan, 1)
GO_enrichemnt_top_wide

In [ ]:
import seaborn as sns
from matplotlib.colors import LogNorm
sns.set(font_scale=1.2)
g = sns.clustermap(GO_enrichemnt_top_wide, norm=LogNorm(), figsize = (15,20), cmap="viridis_r",
                   row_cluster=True, col_cluster=False
                  )
plt.setp(g.ax_heatmap.get_xticklabels(), rotation=30)
plt.savefig(f"{output_path}/Enrichemnt analysis_{gp_compare_type}_top{Top_n_ptw}{gp_bayes_direction}_pathways.png")

In [ ]:
GO_enrichemnt_wide = GO_enrichemnt.pivot(index='Term', columns='comparison_pair', values='Adjusted P-value')

if gp_compare_type == '1to1':
    GO_enrichemnt_wide = GO_enrichemnt_wide.reindex(columns=['1 vs 5','1 vs 2','1 vs 0','1 vs 4','1 vs 3']) #['2 vs 4','2 vs 1','2 vs 3','2 vs 0','2 vs 5']

GO_enrichemnt_wide = GO_enrichemnt_wide.replace(np.nan, 1)

sns.set(font_scale=1.2)
g = sns.clustermap(GO_enrichemnt_wide, norm=LogNorm(), figsize = (18,40), cmap="viridis_r",cbar_pos=(0.1, 0.8, 0.03, 0.1),
                   row_cluster=True, col_cluster=False)
plt.setp(g.ax_heatmap.get_xticklabels(), rotation=30)
plt.savefig(f"{output_path}/Enrichemnt analysis_{gp_compare_type}_{gp_bayes_direction}_pathways.png")

In [ ]:
GO_enrichemnt_wide

## Enrichment of drug targets

In [ ]:
help(gp.enrichr)

In [ ]:
from IPython.display import display

GO_enrichemnt = list()
for comparison_pair in Top_GPs_summary['comparison_pair'].unique():
    print(comparison_pair)
    
    original_string = ''  
    df_subset = Top_GPs_summary[Top_GPs_summary['comparison_pair'] == comparison_pair]
    df_subset.reset_index(drop=True, inplace=True)
    for index, row in df_subset.iterrows():
        original_string = original_string + df_subset['gp_source_genes'][0]
        original_string = original_string + df_subset['gp_target_genes'][0]
    #string_list = re.findall(r"'(.*?)'", original_string)
    string_list = list(set(re.findall(r"'(.*?)'", original_string)))  # Convert to set to remove duplicates
    print(string_list)
    
    enr_KEEG = gp.enrichr(gene_list=string_list,
                          gene_sets=['IDG_Drug_Targets_2022'],#'GO_Biological_Process_2023','MSigDB_Hallmark_2020','KEGG_2021_Human'
                          organism='Human').results
    #'Reactome_2022', KEGG_2021_Human IDG_Drug_Targets_2022' 'MSigDB_Hallmark_2020','MSigDB_Oncogenic_Signatures', 'Metabolomics_Workbench_Metabolites_2022',

    summary = enr_KEEG[enr_KEEG['Adjusted P-value'] < 0.05][['Term', 'Adjusted P-value','Gene_set','Overlap','Genes']]
    summary['comparison_pair'] = comparison_pair
    
    GO_enrichemnt.append(summary)

GO_enrichemnt = pd.concat(GO_enrichemnt)

# remove duplicate terms
GO_enrichemnt= GO_enrichemnt[GO_enrichemnt['Adjusted P-value'] < 0.05]
GO_enrichemnt = GO_enrichemnt.sort_values(by=['comparison_pair', 'Adjusted P-value'], ascending=[True, True])
GO_enrichemnt = GO_enrichemnt.drop_duplicates(subset='Term', keep='first')  # Keep the first occurrence of each unique value in 'term'
GO_enrichemnt.reset_index(drop=True, inplace=True)

GO_enrichemnt_top = GO_enrichemnt.groupby('comparison_pair').head(Top_n_ptw)
GO_enrichemnt_top.reset_index(drop=True, inplace=True)

GO_enrichemnt.to_csv(f"{output_path}/Enrichment_{gp_compare_type}_{gp_bayes_direction}_drugtargets.csv", index = False)
GO_enrichemnt_top.to_csv(f"{output_path}/Enrichment_{gp_compare_type}_top{Top_n_ptw}pathway_{gp_bayes_direction}_drugtargets.csv", index = False)

GO_enrichemnt_top_wide = GO_enrichemnt_top.pivot(index='Term', columns='comparison_pair', values='Adjusted P-value')
if gp_compare_type == '1to1':
    GO_enrichemnt_top_wide = GO_enrichemnt_top_wide.reindex(columns=['1 vs 5','1 vs 2','1 vs 0','1 vs 4','1 vs 3']) #['2 vs 4','2 vs 1','2 vs 3','2 vs 0','2 vs 5']
GO_enrichemnt_top_wide = GO_enrichemnt_top_wide.replace(np.nan, 1)
GO_enrichemnt_top_wide
sns.set(font_scale=1.2)
g = sns.clustermap(GO_enrichemnt_top_wide, norm=LogNorm(), figsize = (15,20), cmap="viridis_r",
                   row_cluster=True, col_cluster=False
                  )
plt.setp(g.ax_heatmap.get_xticklabels(), rotation=30)
plt.savefig(f"{output_path}/Enrichemnt analysis_{gp_compare_type}_top{Top_n_ptw}{gp_bayes_direction}_drugtargets.png")

GO_enrichemnt_wide = GO_enrichemnt.pivot(index='Term', columns='comparison_pair', values='Adjusted P-value')
if gp_compare_type == '1to1':
    GO_enrichemnt_wide = GO_enrichemnt_wide.reindex(columns=['1 vs 5','1 vs 2','1 vs 0','1 vs 4','1 vs 3'])  #['2 vs 4','2 vs 1','2 vs 3','2 vs 0','2 vs 5']
GO_enrichemnt_wide = GO_enrichemnt_wide.replace(np.nan, 1)
sns.set(font_scale=1.2)
g = sns.clustermap(GO_enrichemnt_wide, norm=LogNorm(), figsize = (18,40), cmap="viridis_r",cbar_pos=(0.1, 0.8, 0.03, 0.1),
                   row_cluster=True, col_cluster=False)
plt.setp(g.ax_heatmap.get_xticklabels(), rotation=30)
plt.savefig(f"{output_path}/Enrichemnt_analysis_{gp_compare_type}_{gp_bayes_direction}_drugtargets.png")


## Enrichment of GWAS genes

In [ ]:
from IPython.display import display

GO_enrichemnt = list()
for comparison_pair in Top_GPs_summary['comparison_pair'].unique():
    print(comparison_pair)
    
    original_string = ''  
    df_subset = Top_GPs_summary[Top_GPs_summary['comparison_pair'] == comparison_pair]
    df_subset.reset_index(drop=True, inplace=True)
    for index, row in df_subset.iterrows():
        original_string = original_string + df_subset['gp_source_genes'][0]
        original_string = original_string + df_subset['gp_target_genes'][0]
    #string_list = re.findall(r"'(.*?)'", original_string)
    string_list = list(set(re.findall(r"'(.*?)'", original_string)))  # Convert to set to remove duplicates
    print(string_list)
    
    enr_KEEG = gp.enrichr(gene_list=string_list,
                          gene_sets=['GWAS_Catalog_2023','UK_Biobank_GWAS_v1'],#'GO_Biological_Process_2023','MSigDB_Hallmark_2020','KEGG_2021_Human'
                          organism='Human').results
    #'Reactome_2022', KEGG_2021_Human IDG_Drug_Targets_2022' 'MSigDB_Hallmark_2020','MSigDB_Oncogenic_Signatures', 'Metabolomics_Workbench_Metabolites_2022',

    summary = enr_KEEG[enr_KEEG['Adjusted P-value'] < 0.05][['Term', 'Adjusted P-value','Gene_set','Overlap','Genes']]
    summary['comparison_pair'] = comparison_pair
    
    GO_enrichemnt.append(summary)

GO_enrichemnt = pd.concat(GO_enrichemnt)

# remove duplicate terms
GO_enrichemnt= GO_enrichemnt[GO_enrichemnt['Adjusted P-value'] < 0.05]
GO_enrichemnt = GO_enrichemnt.sort_values(by=['comparison_pair', 'Adjusted P-value'], ascending=[True, True])
GO_enrichemnt = GO_enrichemnt.drop_duplicates(subset='Term', keep='first')  # Keep the first occurrence of each unique value in 'term'
GO_enrichemnt.reset_index(drop=True, inplace=True)

GO_enrichemnt_top = GO_enrichemnt.groupby('comparison_pair').head(Top_n_ptw)
GO_enrichemnt_top.reset_index(drop=True, inplace=True)

GO_enrichemnt.to_csv(f"{output_path}/Enrichment_{gp_compare_type}_{gp_bayes_direction}_GWAS.csv", index = False)
GO_enrichemnt_top.to_csv(f"{output_path}/Enrichment_{gp_compare_type}_top{Top_n_ptw}pathway_{gp_bayes_direction}_GWAS.csv", index = False)

GO_enrichemnt_top_wide = GO_enrichemnt_top.pivot(index='Term', columns='comparison_pair', values='Adjusted P-value')
if gp_compare_type == '1to1':
    GO_enrichemnt_top_wide = GO_enrichemnt_top_wide.reindex(columns=['1 vs 5','1 vs 2','1 vs 0','1 vs 4','1 vs 3'])
GO_enrichemnt_top_wide = GO_enrichemnt_top_wide.replace(np.nan, 1)
GO_enrichemnt_top_wide
sns.set(font_scale=1.2)
g = sns.clustermap(GO_enrichemnt_top_wide, norm=LogNorm(), figsize = (10,15), cmap="viridis_r",
                   row_cluster=True, col_cluster=False
                  )
plt.setp(g.ax_heatmap.get_xticklabels(), rotation=30)
plt.savefig(f"{output_path}/Enrichemnt analysis_{gp_compare_type}_top{Top_n_ptw}{gp_bayes_direction}_GWAS.png")

GO_enrichemnt_wide = GO_enrichemnt.pivot(index='Term', columns='comparison_pair', values='Adjusted P-value')
if gp_compare_type == '1to1':
    GO_enrichemnt_wide = GO_enrichemnt_wide.reindex(columns=['1 vs 5','1 vs 2','1 vs 0','1 vs 4','1 vs 3'])
GO_enrichemnt_wide = GO_enrichemnt_wide.replace(np.nan, 1)
sns.set(font_scale=1.2)
g = sns.clustermap(GO_enrichemnt_wide, norm=LogNorm(), figsize = (18,30), cmap="viridis_r",cbar_pos=(0.1, 0.8, 0.03, 0.1),
                   row_cluster=True, col_cluster=False)
plt.setp(g.ax_heatmap.get_xticklabels(), rotation=30)
plt.savefig(f"{output_path}/Enrichemnt_analysis_{gp_compare_type}_{gp_bayes_direction}_GWAS.png")


## Enrichment of Metabolomics

In [ ]:
#'GWAS_Catalog_2023','UK_Biobank_GWAS_v1','GO_Biological_Process_2023', 'MSigDB_Hallmark_2020','Reactome_2022', KEGG_2021_Human IDG_Drug_Targets_2022'  'MSigDB_Oncogenic_Signatures',  'HMDB_Metabolites','Metabolomics_Workbench_Metabolites_2022',

In [ ]:
from IPython.display import display

GO_enrichemnt = list()
for comparison_pair in Top_GPs_summary['comparison_pair'].unique():
    print(comparison_pair)
    
    original_string = ''  
    df_subset = Top_GPs_summary[Top_GPs_summary['comparison_pair'] == comparison_pair]
    df_subset.reset_index(drop=True, inplace=True)
    for index, row in df_subset.iterrows():
        original_string = original_string + df_subset['gp_source_genes'][0]
        original_string = original_string + df_subset['gp_target_genes'][0]
    #string_list = re.findall(r"'(.*?)'", original_string)
    string_list = list(set(re.findall(r"'(.*?)'", original_string)))  # Convert to set to remove duplicates
    print(string_list)
    
    enr_KEEG = gp.enrichr(gene_list=string_list,
                          gene_sets=['Metabolomics_Workbench_Metabolites_2022'],#'GO_Biological_Process_2023','MSigDB_Hallmark_2020','KEGG_2021_Human'
                          organism='Human').results
    #'Reactome_2022', KEGG_2021_Human IDG_Drug_Targets_2022' 'MSigDB_Hallmark_2020','MSigDB_Oncogenic_Signatures', 'Metabolomics_Workbench_Metabolites_2022',

    summary = enr_KEEG[enr_KEEG['Adjusted P-value'] < 0.05][['Term', 'Adjusted P-value','Gene_set','Overlap','Genes']]
    summary['comparison_pair'] = comparison_pair
    
    GO_enrichemnt.append(summary)


GO_enrichemnt = pd.concat(GO_enrichemnt)

# remove duplicate terms
GO_enrichemnt= GO_enrichemnt[GO_enrichemnt['Adjusted P-value'] < 0.05]
GO_enrichemnt = GO_enrichemnt.sort_values(by=['comparison_pair', 'Adjusted P-value'], ascending=[True, True])
GO_enrichemnt = GO_enrichemnt.drop_duplicates(subset='Term', keep='first')  # Keep the first occurrence of each unique value in 'term'
GO_enrichemnt.reset_index(drop=True, inplace=True)

GO_enrichemnt_top = GO_enrichemnt.groupby('comparison_pair').head(Top_n_ptw)
GO_enrichemnt_top.reset_index(drop=True, inplace=True)

GO_enrichemnt.to_csv(f"{output_path}/Enrichment_{gp_compare_type}_{gp_bayes_direction}_metabo.csv", index = False)
GO_enrichemnt_top.to_csv(f"{output_path}/Enrichment_{gp_compare_type}_top{Top_n_ptw}pathway_{gp_bayes_direction}_metabo.csv", index = False)

GO_enrichemnt_top_wide = GO_enrichemnt_top.pivot(index='Term', columns='comparison_pair', values='Adjusted P-value')
if gp_compare_type == '1to1':
    GO_enrichemnt_top_wide = GO_enrichemnt_top_wide.reindex(columns=['1 vs 5','1 vs 2','1 vs 0','1 vs 4','1 vs 3'])
GO_enrichemnt_top_wide = GO_enrichemnt_top_wide.replace(np.nan, 1)
GO_enrichemnt_top_wide
sns.set(font_scale=1.2)
g = sns.clustermap(GO_enrichemnt_top_wide, norm=LogNorm(), figsize = (10,15), cmap="viridis_r",
                   row_cluster=True, col_cluster=False
                  )
plt.setp(g.ax_heatmap.get_xticklabels(), rotation=30)
plt.savefig(f"{output_path}/Enrichemnt analysis_{gp_compare_type}_top{Top_n_ptw}{gp_bayes_direction}_metabo.png")

GO_enrichemnt_wide = GO_enrichemnt.pivot(index='Term', columns='comparison_pair', values='Adjusted P-value')
if gp_compare_type == '1to1':
    GO_enrichemnt_wide = GO_enrichemnt_wide.reindex(columns=['1 vs 5','1 vs 2','1 vs 0','1 vs 4','1 vs 3'])
GO_enrichemnt_wide = GO_enrichemnt_wide.replace(np.nan, 1)
sns.set(font_scale=1.2)
g = sns.clustermap(GO_enrichemnt_wide, norm=LogNorm(), figsize = (18,30), cmap="viridis_r",cbar_pos=(0.1, 0.8, 0.03, 0.1),
                   row_cluster=True, col_cluster=False)
plt.setp(g.ax_heatmap.get_xticklabels(), rotation=30)
plt.savefig(f"{output_path}/Enrichemnt_analysis_{gp_compare_type}_{gp_bayes_direction}_metabo.png")
